In [1]:
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras import metrics
from tensorflow.keras import backend as K
import numpy as np
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

/Users/stefanroth/anaconda3/envs/poker/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/stefanroth/anaconda3/envs/poker/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/stefanroth/anaconda3/envs/poker/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/stefanroth/anaconda3/envs/p

In [4]:
import itertools
import pandas as pd
import sqlalchemy as sqa
import numpy as np
from tensorflow.keras import models, layers
from tensorflow.keras.layers import Input

DB = f'postgres://localhost/{"poker"}'
ENGINE = sqa.create_engine(DB)

max_features = 53
max_len = 7
vector_size = 4

In [41]:
df = pd.read_sql_table("results", con=ENGINE)

In [42]:
df = df.loc[[0]]
df.head()

,index,action,active_0,active_1,active_2,active_3,active_4,bid,bid_0,bid_1,...,community3,community4,community5,game,hand1,hand2,player,position,round,reward
0,0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,100.0,200.0,...,0.0,0.0,0.0,1.0,33.0,15.0,2.0,2.0,1.0,0.0


In [6]:
y = np.array(df['action'])

In [7]:
rewards = np.array(df['reward'])

In [43]:
cards = np.array(df[['hand1', 'hand2', 'community1', 'community2', 'community3', 'community4', 'community5']].loc[0])
for i in range(1, df.shape[0]):
    cards = np.vstack((cards, np.array(df[['hand1', 'hand2', 'community1', 'community2', 'community3', 'community4', 'community5']].loc[i])))
cards

array([33., 15.,  0.,  0.,  0.,  0.,  0.])

In [9]:
cards.shape

(201, 7)

In [11]:
rest = np.array(df[['position', 'round', 'active_0', 'active_1', 'active_2', 'active_3', 'active_4', 'bid', 'bid_0', 'bid_1', 'bid_2', 'bid_3', 'bid_4']].loc[0])
for i in range(1, df.shape[0]):
    rest = np.vstack((rest, np.array(df[['position', 'round', 'active_0', 'active_1', 'active_2', 'active_3', 'active_4', 'bid', 'bid_0', 'bid_1', 'bid_2', 'bid_3', 'bid_4']].loc[i])))
rest

array([[2.0e+00, 1.0e+00, 1.0e+00, ..., 0.0e+00, 0.0e+00, 0.0e+00],
       [3.0e+00, 1.0e+00, 1.0e+00, ..., 0.0e+00, 0.0e+00, 0.0e+00],
       [4.0e+00, 1.0e+00, 1.0e+00, ..., 0.0e+00, 0.0e+00, 0.0e+00],
       ...,
       [2.0e+00, 3.0e+00, 1.0e+00, ..., 4.0e+02, 8.0e+02, 1.8e+03],
       [0.0e+00, 3.0e+00, 0.0e+00, ..., 4.0e+02, 8.0e+02, 1.8e+03],
       [2.0e+00, 3.0e+00, 1.0e+00, ..., 4.0e+02, 8.0e+02, 1.8e+03]])

In [12]:
rest.shape

(201, 13)

Create a model:

In [13]:
# Card embeddings
card_input = Input(shape=cards.shape[1])
card_output = layers.Embedding(max_features, vector_size, input_length=max_len)(card_input)
card_output = layers.Flatten()(card_output)

# Other information
state_input = Input(shape=rest.shape[1])

#Merge and add dense layer
merge_layer = layers.concatenate([state_input, card_output])
x = layers.Dense(64, activation='relu')(merge_layer)
x = layers.Dense(32, activation='relu')(x)
main_output = layers.Dense(3, activation='softmax')(x)

# Define model with two inputs
model = models.Model(inputs=[card_input, state_input], outputs=[main_output])

#model.compile(
#    optimizer='rmsprop',
#    loss='sparse_categorical_crossentropy',
#    metrics=['accuracy']
#)

W0813 11:25:16.370150 4377101760 deprecation.py:506] From /Users/stefanroth/anaconda3/envs/poker/lib/python3.7/site-packages/tensorflow/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0813 11:25:16.412194 4377101760 deprecation.py:506] From /Users/stefanroth/anaconda3/envs/poker/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [14]:
#    action_prob_ph = self.model.output
action_prob = K.sum(model.output)
action_onehot_ph = K.placeholder(shape=(None, 3),
                                         name="action_onehot")
discount_reward_ph = K.placeholder(shape=(None,),
                                            name="discount_reward")

#action_prob = K.sum(action_prob_ph * action_onehot_ph, axis=1)
log_action_prob = K.log(action_prob)

loss = - log_action_prob * discount_reward_ph
loss = K.mean(loss)

adam = optimizers.Adam()

updates = adam.get_updates(params=model.trainable_weights,
                           #constraints=[],
                           loss=loss)

train_fn = K.function(inputs=[model.input,
                                   action_onehot_ph,
                                   discount_reward_ph],
                           outputs=[action_prob], updates=updates)

model.compile(optimizer=adam, loss='sparse_categorical_crossentropy')

In [44]:
[cards, rest]

[array([33., 15.,  0.,  0.,  0.,  0.,  0.]),
 array([[2.0e+00, 1.0e+00, 1.0e+00, ..., 0.0e+00, 0.0e+00, 0.0e+00],
        [3.0e+00, 1.0e+00, 1.0e+00, ..., 0.0e+00, 0.0e+00, 0.0e+00],
        [4.0e+00, 1.0e+00, 1.0e+00, ..., 0.0e+00, 0.0e+00, 0.0e+00],
        ...,
        [2.0e+00, 3.0e+00, 1.0e+00, ..., 4.0e+02, 8.0e+02, 1.8e+03],
        [0.0e+00, 3.0e+00, 0.0e+00, ..., 4.0e+02, 8.0e+02, 1.8e+03],
        [2.0e+00, 3.0e+00, 1.0e+00, ..., 4.0e+02, 8.0e+02, 1.8e+03]])]

In [25]:
actions = model.predict([cards[0:1], rest[0:1]])
np.squeeze(actions)

array([1.5773242e-16, 1.2698680e-18, 1.0000000e+00], dtype=float32)

In [35]:
np.random.choice(3, p=np.squeeze(actions))

2

In [22]:
model.fit([cards, rest], y, batch_size=10, epochs=10)

Epoch 1/10
201/201 [==============================] - 0s 2ms/sample - loss: 8.5680
Epoch 2/10
201/201 [==============================] - 0s 213us/sample - loss: 3.3222
Epoch 3/10
201/201 [==============================] - 0s 215us/sample - loss: 2.8116
Epoch 4/10
201/201 [==============================] - 0s 226us/sample - loss: 4.5773
Epoch 5/10
201/201 [==============================] - 0s 203us/sample - loss: 8.0140
Epoch 6/10
201/201 [==============================] - 0s 196us/sample - loss: 4.7310
Epoch 7/10
201/201 [==============================] - 0s 198us/sample - loss: 1.5801
Epoch 8/10
201/201 [==============================] - 0s 197us/sample - loss: 1.3348
Epoch 9/10
201/201 [==============================] - 0s 199us/sample - loss: 2.5486
Epoch 10/10
201/201 [==============================] - 0s 204us/sample - loss: 1.8740


In [23]:
K.sum(model.output)

<tf.Tensor 'Sum:0' shape=() dtype=float32>

In [40]:
train_fn = build_train_fn()

In [41]:
history = train([cards, rest], y, rewards)

In [42]:
history.

In [26]:
model.fit([cards, rest], y, batch_size=10, epochs=10)

Epoch 1/10
201/201 [==============================] - 0s 223us/sample - loss: 14.8911 - acc: 0.5224
Epoch 2/10
201/201 [==============================] - 0s 230us/sample - loss: 11.9024 - acc: 0.5622
Epoch 3/10
201/201 [==============================] - 0s 178us/sample - loss: 13.8787 - acc: 0.5373
Epoch 4/10
201/201 [==============================] - 0s 193us/sample - loss: 9.2593 - acc: 0.6070
Epoch 5/10
201/201 [==============================] - 0s 198us/sample - loss: 9.2232 - acc: 0.6617
Epoch 6/10
201/201 [==============================] - 0s 206us/sample - loss: 9.7201 - acc: 0.6119
Epoch 7/10
201/201 [==============================] - 0s 218us/sample - loss: 10.1314 - acc: 0.6070
Epoch 8/10
201/201 [==============================] - 0s 236us/sample - loss: 8.6636 - acc: 0.6119
Epoch 9/10
201/201 [==============================] - 0s 214us/sample - loss: 11.0821 - acc: 0.5622
Epoch 10/10
201/201 [==============================] - 0s 197us/sample - loss: 8.8118 - acc: 0.6468


In [24]:
cards[0].shape

(7,)

In [22]:
[cards[0], rest[0]]

[array([33., 15.,  0.,  0.,  0.,  0.,  0.]),
 array([  2.,   1.,   1.,   1.,   1.,   1.,   1.,   0., 100., 200.,   0.,
          0.,   0.])]

In [16]:
str(np.squeeze(action[0]))

'fold'

In [15]:
pactions = ['fold', 'call', 'raise']
action = []
for i in range(actions.shape[0]):
    action.append(np.random.choice(pactions, 1, p=actions[i]))
    
action

[array(['fold'], dtype='<U5'),
 array(['fold'], dtype='<U5'),
 array(['fold'], dtype='<U5'),
 array(['fold'], dtype='<U5'),
 array(['fold'], dtype='<U5'),
 array(['call'], dtype='<U5'),
 array(['raise'], dtype='<U5'),
 array(['call'], dtype='<U5'),
 array(['call'], dtype='<U5')]

In [ ]:
from keras.layers import Merge

left_branch = Sequential()
left_branch.add(layers.Embedding(max_features, vector_size, input_length=max_len))

right_branch = Sequential()
right_branch.add(Dense(32, input_dim=784))

merged = Merge([left_branch, right_branch], mode='concat')

final_model = Sequential()
final_model.add(merged)
final_model.add(Dense(10, activation='softmax'))

In [24]:
model = models.Sequential()

model.add(layers.Embedding(
    max_features, vector_size, input_length=max_len
    # Number of distinct features, size of the word vector, input size
                          ))
state_input = Input(shape=rest.shape)

model.add(layers.Flatten())
model.add(layers.Dense(1, activation='softmax'))
model.summary()

model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

W0810 17:43:26.784821 4741285312 deprecation.py:506] From /Users/stefanroth/anaconda3/envs/poker/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 7, 4)              212       
_________________________________________________________________
flatten (Flatten)            (None, 28)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 29        
Total params: 241
Trainable params: 241
Non-trainable params: 0
_________________________________________________________________


In [103]:
df.columns

Index(['index', 'action', 'active_0', 'active_1', 'active_2', 'active_3',
       'active_4', 'bid', 'bid_0', 'bid_1', 'bid_2', 'bid_3', 'bid_4',
       'community1', 'community2', 'community3', 'community4', 'community5',
       'hand1', 'hand2', 'player', 'position', 'round', 'reward'],
      dtype='object')

In [10]:
rest = df[['position', 'round', 'active_0', 'active_1', 'active_2', 'active_3', 'active_4', 'bid', 'bid_0', 'bid_1', 'bid_2', 'bid_3', 'bid_4'\
          , 'hand1', 'hand2', 'community1', 'community2', 'community3', 'community4', 'community5']]

In [14]:
df.head()

,index,action,active_0,active_1,active_2,active_3,active_4,bid,bid_0,bid_1,...,community2,community3,community4,community5,hand1,hand2,player,position,round,reward
0,0,4.0,1.0,1.0,1.0,1.0,1.0,200.0,100.0,200.0,...,999.0,999.0,999.0,999.0,25.0,22.0,2.0,2.0,1.0,-400.0
1,1,0.0,1.0,1.0,1.0,1.0,1.0,0.0,100.0,200.0,...,999.0,999.0,999.0,999.0,44.0,51.0,3.0,3.0,1.0,0.0
2,2,4.0,1.0,1.0,1.0,0.0,1.0,200.0,100.0,200.0,...,999.0,999.0,999.0,999.0,31.0,35.0,4.0,4.0,1.0,-1000.0
3,3,5.0,1.0,1.0,1.0,0.0,1.0,400.0,100.0,200.0,...,999.0,999.0,999.0,999.0,11.0,42.0,5.0,5.0,1.0,-1600.0
4,4,4.0,1.0,1.0,0.0,1.0,1.0,400.0,200.0,200.0,...,999.0,999.0,999.0,999.0,0.0,8.0,0.0,0.0,1.0,-2000.0


In [19]:
class Agent:
    '''
    The class Agent takes the following parameters as input:
    
    and uses them to decide on his action in a specified game of poker
    '''
    
    def __init__(self, df, vector_size):
        self.dataframe = df
        self.max_features = 53
        self.vector_size = vector_size
        self.max_len = 7
        self.input_card_embedding = ""
        self.input_state = ""
        self.model = ""
        self.action = ""
    
    def create_embedding_input(self):
        '''
        The function create_embedding_input creates the input for the embedding of the card vectors
        '''
        cards = np.array(self.dataframe[['hand1', 'hand2', 'community1', 'community2', 'community3', 'community4', 'community5']].loc[0])
        for i in range(1, df.shape[0]):
            cards = np.vstack((cards, np.array(self.dataframe[['hand1', 'hand2', 'community1', 'community2', 'community3', 'community4', 'community5']].loc[i])))
        
        self.input_card_embedding = cards
        
    
    def create_state_input(self):
        '''
        The function create_state_input creates the input for the neural network apart from the card embeddings
        '''
        state = np.array(df[['position', 'round', 'active_0', 'active_1', 'active_2', 'active_3', 'active_4', 'bid', 'bid_0', 'bid_1', 'bid_2', 'bid_3', 'bid_4'\
          , 'hand1', 'hand2', 'community1', 'community2', 'community3', 'community4', 'community5']].loc[0])
        for i in range(1, df.shape[0]):
            state = np.vstack((rest, np.array(df[['position', 'round', 'active_0', 'active_1', 'active_2', 'active_3', 'active_4', 'bid', 'bid_0', 'bid_1', 'bid_2', 'bid_3', 'bid_4'\
                  , 'hand1', 'hand2', 'community1', 'community2', 'community3', 'community4', 'community5']].loc[i])))
        
        self.input_state = rest
    
    
    def build_model(self):
        '''
        The function build_model sets up the basic structure for the model that is going to be trained.
        '''
        # Card embeddings
        card_input = Input(shape=self.input_card_embedding.shape[1])
        card_output = layers.Embedding(self.max_features, self.vector_size, input_length=self.input_length)(card_input)
        card_output = layers.Flatten()(card_output)

        # Other information
        state_input = Input(shape=self.state_input.shape[1])

        #Merge and add dense layer
        merge_layer = layers.concatenate([state_input, card_output])
        x = layers.Dense(64, activation='relu')(merge_layer)
        x = layers.Dense(32, activation='relu')(x)
        main_output = layers.Dense(3, activation='softmax')(x)

        # Define model with two inputs
        self.model = models.Model(inputs=[card_input, state_input], outputs=[main_output])
        
        #    action_prob_ph = self.model.output
        action_prob = K.sum(self.model.output)
        action_onehot_ph = K.placeholder(shape=(None, 3),
                                                 name="action_onehot")
        discount_reward_ph = K.placeholder(shape=(None,),
                                                    name="discount_reward")

        #action_prob = K.sum(action_prob_ph * action_onehot_ph, axis=1)
        log_action_prob = K.log(action_prob)

        loss = - log_action_prob * discount_reward_ph
        loss = K.mean(loss)

        adam = optimizers.Adam()

        updates = adam.get_updates(params=model.trainable_weights,
                                   #constraints=[],
                                   loss=loss)

        train_fn = K.function(inputs=[self.model.input,
                                           action_onehot_ph,
                                           discount_reward_ph],
                                   outputs=[action_prob], updates=updates)

        self.model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

#        model.compile(
#            optimizer='rmsprop',
#            loss='sparse_categorical_crossentropy',
#            metrics=['accuracy']
#        )
        
    def chose_action(self, state: list):
        self.action = np.squeeze(np.random.choice(self.model.predict(state)))
        # don't forget to introduce baseline
        

In [87]:
agent = Agent(df)

In [88]:
agent.create_embedding_input()

In [89]:
agent.input_card_embedding

array([[ 25.,  22., 999., ..., 999., 999., 999.],
       [ 44.,  51., 999., ..., 999., 999., 999.],
       [ 31.,  35., 999., ..., 999., 999., 999.],
       ...,
       [ 49.,  34., 999., ..., 999., 999., 999.],
       [ 31.,  46., 999., ..., 999., 999., 999.],
       [  8.,   1., 999., ..., 999., 999., 999.]])

In [4]:
df.hand1[0], df.hand2[1]

(25.0, 51.0)

Let us look at the community cards:

In [8]:
for i in range(df.shape[0]):
    if len(df.community[i]) > 2:
        print(df.community[i])

{"(q,c)","(5,d)","(7,c)"}
{"(q,c)","(5,d)","(7,c)"}
{"(q,c)","(5,d)","(7,c)","(j,d)"}
{"(q,c)","(5,d)","(7,c)","(j,d)"}
{"(q,c)","(5,d)","(7,c)","(j,d)"}
{"(3,d)","(t,h)","(a,c)"}
{"(3,d)","(t,h)","(a,c)"}
{"(3,d)","(t,h)","(a,c)","(q,c)"}
{"(3,d)","(t,h)","(a,c)","(q,c)"}
{"(3,h)","(7,s)","(9,c)"}
{"(3,h)","(7,s)","(9,c)"}
{"(3,h)","(7,s)","(9,c)"}
{"(3,h)","(7,s)","(9,c)"}
{"(3,h)","(7,s)","(9,c)"}
{"(3,h)","(7,s)","(9,c)"}
{"(3,h)","(7,s)","(9,c)"}
{"(3,h)","(7,s)","(9,c)"}
{"(3,h)","(7,s)","(9,c)"}
{"(3,h)","(7,s)","(9,c)"}
{"(3,h)","(7,s)","(9,c)"}
{"(3,h)","(7,s)","(9,c)"}
{"(3,h)","(7,s)","(9,c)"}
{"(3,h)","(7,s)","(9,c)"}
{"(3,h)","(7,s)","(9,c)"}
{"(3,h)","(7,s)","(9,c)","(k,d)"}
{"(3,h)","(7,s)","(9,c)","(k,d)"}
{"(3,h)","(7,s)","(9,c)","(k,d)"}
{"(3,h)","(7,s)","(9,c)","(k,d)"}
{"(3,h)","(7,s)","(9,c)","(k,d)"}
{"(3,h)","(2,c)","(4,h)"}
{"(3,h)","(2,c)","(4,h)"}
{"(3,h)","(2,c)","(4,h)","(a,c)"}
{"(3,h)","(2,c)","(4,h)","(a,c)"}
{"(k,d)","(2,c)","(5,d)"}
{"(k,d)","(2,c)","(5